# Introduction
This kernel has been created by the [Information Systems Lab](http://islab.uom.gr) at the University of Macedonia, Greece for the needs of the elective course Special Topics of Information Systems I at the [Business Administration](http://www.uom.gr/index.php?tmima=2&categorymenu=2) department of the University of Macedonia, Greece.

In this Instacart Notebook, we answer business questions regarding how consumers behave towards a specific product and thus we calculate variables that describe each product. 

To achieve this, we follow three steps:
1. Create a new DataFrame, namely **prd**, that contains data from multiple CSV files
2. Produce supportive variables with aggregated values from the prd DataFrame
3. Calculate variables that describe products

# Business Insights
In this notebook you will explore Instacart data in order to answer the following business questions:
* Do users frequently reorder a product? i.e. what is the reorder probability of a product? More specific questions incude:
 - What is the number of unique customers of a product?
 - How many customers buy a product only once (one-shot products)? 
 - What is the one-shot ratio of a product? (This ratio is related to the reorder probability of a product)
 - What is the mean one-shot ratio of an aisle?

# Python Skills
* Merge two DataFrame with inner, left, right join
* Perform a .groupby( ) on data of a DataFrame that meet a condition
* Create a new column to an already existing DataFrame
* Convert and index of DataFrame into a column
* Divide columns element-wise of a data frame
* Check for NaN (Not a Number) values and modify them
* Create a histrogram (distribution plot) of a variable
* Set a column as the index of a DataFrame
* Filter data frames based on a condition
* Select rows of a DataFrame, based on a condition
* Create a new DataFrame, with column(s) of an existing DataFrame
* Create ratios through supportive variables

# Packages 
* pandas: .merge() , .reset_index( ),  .isnull(), .any(), set_index(), 
* matplotlib.pyplot: .hist()
* seaborn: .barplot()

## Import data into Python
We load the required packages:

In [ ]:
import pandas as pd               # for data manipulation
import matplotlib.pyplot as plt   # for plotting 
import seaborn as sns             # an extension of matplotlib for statistical graphics

Moreover, we load the .csv files in DataFrames:

In [ ]:
orders = pd.read_csv('../input/orders.csv' )
products = pd.read_csv('../input/products.csv')
order_products_prior = pd.read_csv('../input/order_products__prior.csv')
aisles = pd.read_csv('../input/aisles.csv')

# 1. Create a DataFrame that contains data from multiple sources 
In this section, we create a new DataFrame that combines the orders, the customers, and the products. Towards this end, we use the following DataFrames:

**orders** which contains the orders made from all customers

In [ ]:
orders.head()

**order_products_prior** which contains the products purchased in each order 

In [ ]:
order_products_prior.head()

We merge these two DataFrames by their matching column, order_id. The inner join (how='inner') keeps only rows where each order_id can be found on both DataFrames. **orders** DataFrame contains both prior and train orders, while **order_products_prior** contains only prior orders. 

As a result, the new DataFrame contains only prior orders.

![inner](https://www.w3schools.com/Sql/img_innerjoin.gif)

In [ ]:
prd = pd.merge(orders, order_products_prior, on='order_id', how='inner')
prd.head(10)

Now that we have a DataFrame that combines both the prior orders and the products purchased on each order, we will get insights for each product.

# 2. Does users frequently reorder a product?  (one-shot ratio)
In order to answer this business question we calculate a ratio that consists of the total number of customers who bought a product only once divided by the total number of customers who bought this product.

![](https://latex.codecogs.com/gif.latex?one\&space;-\&space;shot\&space;ratio\&space;of\&space;a\&space;product\&space;=&space;\frac{customers\&space;who\&space;bought\&space;it\&space;only\&space;once}{number\&space;of\&space;unique\&space;customers})

A high one-shot ratio of a product means that customers tend not to reorder this product. 

To calculate this ratio we have to compute:
* the number of users bought it only once (count)
* the number of unique customers of a product (count)

These variables will be calculated through a supportive variable that indicates how many times each customer bought a product. We start by calculating this supportive variable.

## 2.1  How many times each customer bought a product.
To create this supportive variable, we .groupby( ) the **prd** DataFrame, by the product_id & user_id and we select the column 'order_id' to apply the .count( ) aggregation function. We store the result in a new DataFrame called item.

In [ ]:
item = prd.groupby(['product_id', 'user_id'])[['order_id']].count()
item.columns = ['total']
item.head(10)

## 2.2  How many customers bought a product only once
From the **item** DataFrame we can now calculate both numerator (the number of users who bought a product only once) & denominator (the number of unique customers for a product) of the one-shot ratio.

To get the number of users who bought it only once:
* We select from **item** DataFrame, only these rows where <b>total purchases are equal to one</b>


In [ ]:
item_one = item[item.total==1]
item_one.head()

* From the selected rows, we perform a .groupby( ) for each product_id, we select the previously created column 'total' and we use the aggregation function .count( ) to get how many customers bought each product only once.

In [ ]:
item_one = item_one.groupby('product_id')[['total']].count()
item_one.columns = ['customers_one_shot']

item_one.head(10)

So for example, product 2 has been bought only one time from 70 individual customers.

### 2.2.1 Your Turn 📚📝
Which other aggregation function will yield the same result?

In [ ]:
# Write your answer here
item_one= item[item.total==1].groupby('product_id')[['total']].sum()
item_one.columns = ['customers_one_shot']
item_one.head()

## 2.3 Number of unique customers of a product.
We now perform a groupby on **product_id** and count the users that appear in each product. 

However, **user_id** is passed as index on **item** DataFrame. For this reason, we use the **.reset_index( )** method to convert **user_id** to a column. In our case we use the argument=1 on **.reset_index( )**, as we want only the second index (**user_id**) to be turned into a column.

In [ ]:
#execute command one-time only
item=item.reset_index(1)

In [ ]:
item.head(10)

So now, we can perform our desired .groupby( ) on **item** :

In [ ]:
item_size = item.groupby('product_id')[['user_id']].count()
item_size.columns = ['unique_customers']
item_size.head(10)

From the results we see that for example product 2 has been bought from 78 individual customers.

### 2.3.1 Your Turn 📚📝
How could you create the same variable by using the prd DataFrame & the reorder column?
> Hint: You will need to filter rows of **prd** DataFrame first.

In [ ]:
item_size['unique_customers']= prd[prd.reordered==0].groupby('product_id')[['user_id']].count()
item_size.head(10)

## 2.4 Merge results
Now that we have both parts of our final fraction (ratio), we will merge both DataFrames into one. We select a right join as the **item_size** data frame, keeps all the product that they have been purchased, where the **item_one** keeps only the products that they have been purchased only once.

![right](https://www.w3schools.com/sql/img_rightjoin.gif)

In [ ]:
#                                          COMMENT FOR RIGHT JOIN:
# item_one: may or may not contain observations for each product (may never bought only one time) 
#                         item_size: must have values for every product (bought even once) 
#                                       left or inner joins will lose observations. 

results = pd.merge(item_one, item_size, on='product_id', how='right')
results.head()

As a result we have a DataFrame that keeps both variables for 49677 products:

In [ ]:
results.shape

### 2.4.1 Your Turn - Use a left join 📚📝
Propose another way to use a left join to merge the DataFrames **item_one** & **item_size**.  Your new DataFrame should have the same .shape (49677, 2) as the previous DataFrame:

In [ ]:
# Write your answer here
results = pd.merge(item_size, item_one, on='product_id', how='left')
results.head()

In [ ]:
results.shape

## 2.6 Create the one-shot ratio
Now that we have merged the two count variables,  we can create a new column which will keep our desired ratio. 
To create a new column to an already existing DataFrame you can simply write the name of the DataFrame and the name of the new column in brackets.

In [ ]:
results['one_shot_ratio'] = results['customers_one_shot']/results['unique_customers']
results.head()

# 4. Further exploration of results DataFrame
Now that we have both the one-shot ratio & reorder_times_product for each product, we show how we:
1. Find and fill NaN values
2. Visualize the one-shot ratio
3. Analyze the products with one-shot ratio = 1
4. Get the name of the products
5. Get the products with highest and lowest one-shot ratio
6. Find the aisle with highest mean one-shot ratio
7. Create visualizations for the products and the aisles with the highest ratio
9. Get the 10 products with the highest reorder_times_product

## 4.1 Find and fill NaN values
There may be cases where none customer bought a product only one time. In this case, the corresponding variable after merge will have a NaN value.

Let's check the above scenario. To retrieve the rows with a NaN in a particular column, we can use the following code:
>df[df['Col1'].isnull()]

So we modify the previous code for the column customers_bought_once:

In [ ]:
results[results.customers_one_shot.isnull()].head(10)

In [ ]:
results[results.customers_one_shot.isnull()].shape

Which means that 90 products have never been bought only once.
As NaN values, means absence of an observation for each product, we will use the .fillna(0) method so we can convert all NaN values to zero.

In [ ]:
results = results.fillna(0)
results.head()

If we check again for NaN values on results DataFrame we will get no results.

In [ ]:
results[results.customers_one_shot.isnull()]

## 4.2 Visualize the one-shot ratio
In order to visualize the one-shot ratio in a histogram, we use the .hist( ) method of matplotlib package.
Argument bins=100, creates a histrogram of 100 bins (bin of 0,01 for range 0 to 1) as ratio is a continuous variable.

In [ ]:
plt.figure(figsize=(15,5))

plt.hist(results.one_shot_ratio, bins=100)

plt.xlabel('Probability', size=10)
plt.ylabel('Number of products')
plt.title('The distribution of one-shot ratio', size=10)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

## 4.3 Analyze the products with One-Shot ratio = 1
Interestingly many product have one-shot ratio = 1 meaning that these products have never been reordered. In this sub-section we explore these products.

In [ ]:
no_reorder = results[results.one_shot_ratio==1]
no_reorder.head(10)

There are actually 4372 with no reorders.

In [ ]:
no_reorder.shape[0]

Which are actually 8,8% of total products 

In [ ]:
no_reorder.shape[0]/results.shape[0]*100

If you check again on no_reorder DataFrame it seems that products with one-shot ratio = 1  have few unique customers. 

To have a broader view on products with one-shot ratio = 1, we create a histrogram for the number of unique customers

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(no_reorder[no_reorder.one_shot_ratio==1].unique_customers, bins=90)
plt.show()

So Indeed, most of the products that are bought only one time tend to have less than 20 unique customers.

## 4.4 Get the name of the products
Now we will merge the results with the **products** DataFrame. The joining key is the product_id and we select a left join, as we want to keep all these products, that we have created a one-shot ratio for them.
![left](https://www.w3schools.com/sql/img_leftjoin.gif)

In [ ]:
# merge with products, to get the names, aisle, department of the products
results_name = pd.merge(results, products[['product_id', 'product_name', 'aisle_id']], on='product_id', how='left')
results_name.head()

As each row describe a single product, we select product_id to be the main key for each row; with .set_index( ) method we select product_id to be the index for results_name

In [ ]:
results_name = results_name.set_index('product_id')
results_name.head()

## 4.5 Get the products with highest and lowest ratio
In the coming example we select the products with the highest and lowest one-shot ratio. We select to filter (limit) our results to only these products that have more than 75 unique customers. Products with very few customers cannot provide valid data.
> We chose to be 75 unique customers, as many products with less than 75 unique customers, had one-shot ratio equal to one.

In [ ]:
# get the products with highest one_time_ratio - we need an estimation for no. unique_customers
results_asc = results_name.sort_values('one_shot_ratio', ascending=False)
results_asc = results_asc[results_asc.unique_customers > 75]
results_asc.head(10)

In [ ]:
# get the products with lowest one_time_ratio
results_des = results_name.sort_values('one_shot_ratio', ascending=True)
results_des = results_des[results_des.unique_customers > 75]
results_des.head(10)

## 4.6  Aisles with high mean one-shot ratio
In this step, we .groupby( ) products by their aisle_id. With aggregation function .mean( ) we get the average of one_shot_ratio for each aisle.


In [ ]:
aisle_ratio = results_asc.groupby('aisle_id')[['one_shot_ratio']].mean()
aisle_ratio.columns = ['mean_one_shot_ratio_of_aisle']
aisle_ratio.head()

With .sort_values( ) we get the aisles with the highest mean of one-shot ratio.

In [ ]:
aisle_ratio = aisle_ratio.sort_values('mean_one_shot_ratio_of_aisle', ascending=False)
aisle_ratio.head()

And now we get the actual names of the aisles from aisles data frame.

Here we use a left join as we want to keep all the aisles that we have metrics for them, and just join on them the available information of aisles data frame.

In [ ]:
#get the name of aisles with merge
aisle_ratio_names = pd.merge(aisle_ratio, aisles, on='aisle_id', how='left')
#set aisle_id as index
aisle_ratio_names = aisle_ratio_names.set_index('aisle_id')
aisle_ratio_names.head(15)

## 4.7 Create visualizations for the products and the aisles with the highest ratio
At this stage we create two plots; one for the products & one for the aisles with the highest ratio.

First we trim **results_asc** (the DataFrame that keeps the one-shot ratio fo each product) to keep only the top 10.

In [ ]:
#perform visualization of the products with the lowest one time ratio
results_asc_top = results_asc.iloc[0:10]
results_asc_top

And now we create a barplot with top products.

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(results_asc_top.one_shot_ratio, results_asc_top.product_name)
# add label to x-axis
plt.xlabel('Ratio (ordered once/total orders)', size=15)
# keep y-axis free of label
plt.ylabel('  ')
#put a title
plt.title('Top 10 products that are ordered only once', size=15)

#make tick locations (titles) more visible
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

#we set the range of y-axis to a bit lower from the lowest probability and a bit higher from the higest probability
plt.xlim(0.98, 1.001)
plt.show()

In the same fashion, we create a plot for the aisles with the highest mean of one-shot ratio. The information for the aisles is stored in the **aisle_ratio_names** DataFrame.

In [ ]:
aisle_top = aisle_ratio_names.iloc[:10]
aisle_top

In [ ]:
plt.figure(figsize=(12,8))

sns.barplot(aisle_top.aisle, aisle_top.mean_one_shot_ratio_of_aisle)

# keep x-axis free of label 
plt.xlabel(' ')
plt.ylabel('Mean one-shot ratio of aisle', size=15)
plt.title('Top 10 aisles that their products tend to ordered only once', size=15)

#make tick locations (titles) more visible, rotate axis of xticks
plt.xticks(fontsize=15, rotation=90)
plt.yticks(fontsize=15)

#we set the range of y-axis to a bit lower from the lowest probability and a bit higher from the higest probability
plt.ylim(0.8, 0.9)
plt.show()

# 5 Merge the final variables with prd DataFrame
In the final chapter, we show how we can merge the final variable created from Section 2 to the initial **prd** DataFrame that holds data for both the orders and products purchased. 
## 5.1 Create a DataFrame from results DataFrame
Here, we select to keep only the column 'one_shot_ratio'  and store it a new DataFrame called **results_trimmed**


In [ ]:
results_trimmed = results[['one_shot_ratio']]
results_trimmed.head()

### 5.1.2 Merge one_shot_trimmed with prd 
And now we join the results_trimmed to the initial prd data frame. We use a left join to keep all the orders and products on the prd data frame.
![left](https://www.w3schools.com/sql/img_leftjoin.gif)

Note that we would not be able to merge the DataFrames, if we didn't have as index the product_id on results_trimmed.

In [ ]:
# get the results to the original merged data frame 
prd = pd.merge(prd , results_trimmed, on='product_id', how='left')
prd.head()